# The Battle of Neighborhoods in North York, Canada

In [1]:
! pip install folium

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Importing DATA from Wikipedia and reading it using pandas

In [3]:
filename="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df=pd.read_html(filename)
df=df[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df.replace("Not assigned",np.nan,inplace=True)
df=df[df["Borough"].notna()]
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
df.shape

(103, 3)

In [6]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
df1=pd.merge(geo_df,df,on="Postal Code")
df2=df1[["Postal Code","Borough","Neighbourhood","Latitude","Longitude"]]
df2.rename(columns={"Neighbourhood":"Neighborhood"},inplace=True)
df=df2
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Getting just North York Borough


In [7]:
ny_df=df[df["Borough"]=="North York"].reset_index(drop=True)
ny_df.head(7)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493
5,M2N,North York,"Willowdale, Willowdale East",43.770120,-79.408493
6,M2P,North York,York Mills West,43.752758,-79.400049


## Using the FourSquare API to access neighborhood venue data

In [8]:
CLIENT_ID = 'TPSH5D2XWAR0WR20M0Y35ITLF5ILJEXUMVDXVJ4CBLGO4S0J' # your Foursquare ID
CLIENT_SECRET = 'JURU3R4PDYQIEGZOEZ1UR1F1A2SD3YHLDZRZDC10RZVPYKNV' # your Foursquare Secret
VERSION = '20201124' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TPSH5D2XWAR0WR20M0Y35ITLF5ILJEXUMVDXVJ4CBLGO4S0J
CLIENT_SECRET:JURU3R4PDYQIEGZOEZ1UR1F1A2SD3YHLDZRZDC10RZVPYKNV


In [9]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
ny_venues = getNearbyVenues(names=ny_df['Neighborhood'],
                                   latitudes=ny_df['Latitude'],
                                   longitudes=ny_df['Longitude']
                                  )


Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Humberlea, Emery


In [11]:
ny_venues.head(3)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant


In [12]:
ny_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Don Mills,25,25,25,25,25,25
Downsview,17,17,17,17,17,17
"Fairview, Henry Farm, Oriole",71,71,71,71,71,71
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,2,2,2,2,2,2


## One Hot Encoding

In [13]:
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
#ny_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
ny_onehot.insert(loc=0, column='Neighborhood', value=ny_venues['Neighborhood'] )
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Locksmith,Lounge,Luggage Store,Martial Arts School,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.0,0.0,0.095238,0.0,0.000000,0.00,0.00,0.0,0.047619,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.047619,0.0,0.00,0.095238,0.000000,0.0,0.0,0.0,0.047619,0.0,0.0,0.00,0.047619,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.047619,0.0,0.047619,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.047619,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.047619,0.0,0.000000,0.0,0.0,0.047619,0.047619,0.0,0.00,0.0,0.000000,0.0,0.047619,0.0,0.047619,0.0,0.047619,0.0,0.000000,0.00,0.0,0.047619,0.0,0.047619,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.0,0.0,0.250000,0.0,0.000000,0.00,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.250000,0.00,0.250000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.045455,0.00,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.045455,0.045455,0.00,0.000000,0.0,0.00,0.090909,0.045455,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.045455,0.045455,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.045455,0.0,0.090909,0.00,0.045455,0.045455,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.045455,0.045455,0.0,0.00,0.0,0.045455,0.0,0.045455,0.0,0.090909,0.0,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.0,0.045455,0.0,0.00,0.045455,0.0,0.0,0.0,0.0,0.0
3,Don Mills,0.0,0.000000,0.000000,0.04,0.0,0.04,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.08,0.04,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.040000,0.04,0.040000,0.0,0.04,0.080000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.04,0.000000,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.12,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.040000,0.08,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0

In [14]:
ny_grouped.shape

(19, 107)

## Returning the ten most common venues for each neighborhood

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Ice Cream Shop,Middle Eastern Restaurant,Mobile Phone Shop,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Grocery Store,Greek Restaurant,Indian Restaurant,Juice Bar,Liquor Store,Locksmith,Comfort Food Restaurant
3,Don Mills,Gym,Japanese Restaurant,Coffee Shop,Beer Store,Bike Shop,Italian Restaurant,Dim Sum Restaurant,Discount Store,Restaurant,Sandwich Place
4,Downsview,Grocery Store,Park,Speakeasy,Home Service,Business Service,Shopping Mall,Baseball Field,Hotel,Liquor Store,Bank
5,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Juice Bar,Women's Store,Food Court,Cosmetics Shop,Shoe Store,Bank
6,Glencairn,Pizza Place,Pub,Bakery,Japanese Restaurant,Park,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
7,Hillcrest Village,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Dog Run,Frozen Yogurt Shop,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
8,Humber Summit,Pizza Place,Furniture / Home Store,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
9,"Humberlea, Emery",Baseball Field,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


## Clustering the North York Neighborhhods using K-Means Clustering

In [17]:
# set number of clusters
kclusters = 6

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40]


array([4, 4, 4, 4, 4, 4, 5, 4, 3, 2, 4, 5, 4, 0, 4, 4, 4, 0, 1],
      dtype=int32)

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
ny_merged = ny_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged1 = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [19]:
ny_merged1.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,4.0,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Dog Run,Frozen Yogurt Shop,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Juice Bar,Women's Store,Food Court,Cosmetics Shop,Shoe Store,Bank
2,M2K,North York,Bayview Village,43.786947,-79.385975,4.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,1.0,Martial Arts School,Women's Store,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
ny_merged1.head()

ny_merged1.dropna(inplace=True)
ny_merged1["Cluster Labels"]=ny_merged1["Cluster Labels"].astype(int)
ny_merge=ny_merged1
ny_merge.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,4,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Dog Run,Frozen Yogurt Shop,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Juice Bar,Women's Store,Food Court,Cosmetics Shop,Shoe Store,Bank
2,M2K,North York,Bayview Village,43.786947,-79.385975,4,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,1,Martial Arts School,Women's Store,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
5,M2N,North York,"Willowdale, Willowdale East",43.770120,-79.408493,4,Ramen Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Café,Japanese Restaurant,Sushi Restaurant,Hotel,Ice Cream Shop,Bubble Tea Shop
6,M2P,North York,York Mills West,43.752758,-79.400049,0,Park,Convenience Store,Women's Store,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
7,M2R,North York,"Willowdale, Willowdale West",43.782736,-79.442259,4,Grocery Store,Butcher,Pharmacy,Pizza Place,Coffee Shop,Frozen Yogurt Shop,Dessert Shop,Gift Shop,Gas Station,Comfort Food Restaurant
8,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
9,M3B,North York,Don Mills,43.745906,-79.352188,4,Gym,Japanese Restaurant,Coffee Shop,Beer Store,Bike Shop,Italian Restaurant,Dim Sum Restaurant,Discount Store,Restaurant,Sandwich Place
10,M3C,North York,Don Mills,43.725900,-79.340923,4,Gym,Japanese Restaurant,Coffee Shop,Beer Store,Bike Shop,Italian Restaurant,Dim Sum Restaurant,Discount Store,Restaurant,Sandwich Place


In [21]:
neighborhoods_venues_sorted.head()


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Ice Cream Shop,Middle Eastern Restaurant,Mobile Phone Shop,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega
1,4,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Grocery Store,Greek Restaurant,Indian Restaurant,Juice Bar,Liquor Store,Locksmith,Comfort Food Restaurant
3,4,Don Mills,Gym,Japanese Restaurant,Coffee Shop,Beer Store,Bike Shop,Italian Restaurant,Dim Sum Restaurant,Discount Store,Restaurant,Sandwich Place
4,4,Downsview,Grocery Store,Park,Speakeasy,Home Service,Business Service,Shopping Mall,Baseball Field,Hotel,Liquor Store,Bank


## Retrieving the longitude and latitude from the Geolocator

In [22]:
address = 'North York, CA'

geolocator = Nominatim(user_agent="northyork")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


## Using folium to generqate a map with the clusters

In [23]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merge['Latitude'], ny_merge['Longitude'], ny_merge['Neighborhood'], ny_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining each cluster in North York

In [24]:
ny_merge.loc[ny_merge['Cluster Labels'] == 0, ny_merge.columns[[1] + list(range(5, ny_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,0,Park,Convenience Store,Women's Store,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
8,North York,0,Park,Food & Drink Shop,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


In [25]:
ny_merge.loc[ny_merge['Cluster Labels'] == 1, ny_merge.columns[[1] + list(range(5, ny_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,1,Martial Arts School,Women's Store,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [26]:
ny_merge.loc[ny_merge['Cluster Labels'] == 2, ny_merge.columns[[1] + list(range(5, ny_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,2,Baseball Field,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [27]:
ny_merge.loc[ny_merge['Cluster Labels'] == 3, ny_merge.columns[[1] + list(range(5, ny_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North York,3,Pizza Place,Furniture / Home Store,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


In [28]:
ny_merge.loc[ny_merge['Cluster Labels'] == 4, ny_merge.columns[[1] + list(range(5, ny_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Dog Run,Frozen Yogurt Shop,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,North York,4,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Juice Bar,Women's Store,Food Court,Cosmetics Shop,Shoe Store,Bank
2,North York,4,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
5,North York,4,Ramen Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Café,Japanese Restaurant,Sushi Restaurant,Hotel,Ice Cream Shop,Bubble Tea Shop
7,North York,4,Grocery Store,Butcher,Pharmacy,Pizza Place,Coffee Shop,Frozen Yogurt Shop,Dessert Shop,Gift Shop,Gas Station,Comfort Food Restaurant
9,North York,4,Gym,Japanese Restaurant,Coffee Shop,Beer Store,Bike Shop,Italian Restaurant,Dim Sum Restaurant,Discount Store,Restaurant,Sandwich Place
10,North York,4,Gym,Japanese Restaurant,Coffee Shop,Beer Store,Bike Shop,Italian Restaurant,Dim Sum Restaurant,Discount Store,Restaurant,Sandwich Place
11,North York,4,Coffee Shop,Bank,Gas Station,Ice Cream Shop,Middle Eastern Restaurant,Mobile Phone Shop,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega
12,North York,4,Furniture / Home Store,Caribbean Restaurant,Miscellaneous Shop,Metro Station,Massage Studio,Bar,Coffee Shop,Women's Store,Dim Sum Restaurant,Construction & Landscaping
13,North York,4,Grocery Store,Park,Speakeasy,Home Service,Business Service,Shopping Mall,Baseball Field,Hotel,Liquor Store,Bank
